In [ ]:
import sys
import os
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from password.password import *


target_url = "https://www.rakuten-sec.co.jp/ITS/V_ACT_Login.html"
target_list = ["ひふみプラス",
               "AI(人工知能)活用型世界株ファンド(ディープAI)",
               "eMAXIS Slim 先進国株式インデックス",
               "グローバルAIファンド",
               "グローバルAIファンド(為替ヘッジあり)",
               "楽天日本株4.3倍ブル",
               "ベトナム成長株インカムファンド"]

#ログイン画面
browser = webdriver.Chrome()
browser.get(target_url)
id = browser.find_element_by_id("form-login-id")
id.send_keys(ID)
password = browser.find_element_by_id("form-login-pass")
password.send_keys(PASSWORD)
time.sleep(1)
login_button = browser.find_element_by_id("login-btn")
login_button.click()
browser.find_element_by_xpath('//*[@id="gmenu_invest"]/a').click()
time.sleep(1)

#確認画面
try:
    browser.find_element_by_class_name('modal__button--normal').click()
except:
    pass
time.sleep(1)
browser.find_element_by_xpath('//*[@id="show_asset_total_table"]/span/table/tbody/tr/td[3]/span[2]/span[1]/a').click()

#投資信託の画面
tbl_element = browser.find_element_by_id('poss-tbl-sp')
target_dic = {}
trs = tbl_element.find_elements_by_tag_name("tr")

for i in range(1,len(trs)):
    tds = trs[i].find_elements(By.TAG_NAME, "td")
    for j in tds:
        try:
            if str(j.text) in target_list:
                url = {"url": j.find_element_by_css_selector("div.mbody").find_element_by_css_selector("a").get_attribute("href")}
                target_dic[j.text] = url
                print(j.text)
                print(j.find_element_by_css_selector("div.mbody").find_element_by_css_selector("a").get_attribute("href"))
        except:
            pass
        
        
for target in target_list[:1]:
    url = target_dic[target]["url"]
    browser.get(url)  
    browser.find_element_by_xpath('//*[@id="infoPage"]/div[6]/div[2]/p[1]/a').click()
    
    window = browser.window_handles[1]
    browser.switch_to.window(window)
    
    tbl_element = browser.find_element_by_class_name('tbl-data-02')
    trs = tbl_element.find_elements_by_tag_name("tr")
    data_dic = {"data":{}}
    for i in range(1,len(trs)):
        tds = trs[i].find_elements(By.TAG_NAME, "td")
        try:
            data_dic["data"]={}
            data_dic["data"]["基準価額"]=tds[0].text
            data_dic["data"]["前日比"]=tds[1].text
            data_dic["data"]["分配金"]=tds[2].text
            data_dic["data"]["純資産総額"]=tds[3].text
            print(data_dic)
        except:
            pass
        
        

In [ ]:
url

In [3]:
import sys
import os
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from password.password import *

from selenium.webdriver.common.keys import Keys

def adjust_df(df):
    def adjust_dividend(x):
        if x =="---":
            x = 0
        elif "円" in str(x):
            x = float(x.replace("円",""))   
        else:
            x = None
        return x

    def adjust_ratio(x):
        if "円" in str(x):
            x = float(x.replace("円",""))   
        else:
            x = None
        return x

    def adjust_baseprice(x):
        if "円" in str(x):
            x = float(x.replace("円","").replace(",",""))   
        else:
            x = None
        return x

    def adjust_assets(x):
        if "億円" in str(x):
            x = float(x.replace("億円","")) *100000000
        return x

    df["分配金"]= df["分配金"].apply(lambda x: adjust_dividend(x))
    df["前日比"] = df["前日比"].apply(lambda x: adjust_ratio(x))
    df["基準価額"] = df["基準価額"].apply(lambda x: adjust_baseprice(x))
    df["日時"] = pd.to_datetime(df["日時"], format='%Y年%m月%d日')
    df["純資産総額"]= df["純資産総額"].apply(lambda x: adjust_assets(x))
    
    return df

def get_data_investment(browser,name_investment,pages=1000):
     #プルダウンでパラーメータ変更
    try:
        element_year = browser.find_element_by_class_name('form-select-01')
        year_select_element = Select(element_year)
        year_select_element.select_by_value('2017')
        browser.find_element_by_class_name('btn-view').click()
    except:
        print("Cannot select 2017")

    #検索
    
    list_data = []
    try:
        for i in range(pages):
            tbl_element = browser.find_element_by_class_name('tbl-data-02')
            trs = tbl_element.find_elements_by_tag_name("tr")

            for i in range(1,len(trs)):
                tds = trs[i].find_elements(By.TAG_NAME, "td")
                ths = trs[i].find_elements(By.TAG_NAME, "th")
                try:
                    dic_data = {}
                    dic_data["日時"] = ths[0].text
                    dic_data["基準価額"]=tds[0].text
                    dic_data["前日比"]=tds[1].text
                    dic_data["分配金"]=tds[2].text
                    dic_data["純資産総額"]=tds[3].text
                    list_data.append(dic_data)
                except:
                    pass
            browser.find_element_by_xpath('//*[@id="str-main"]/form/ul/li[3]').click()
    except:
        print("No more pages")

    df = pd.DataFrame(list_data)
    df = adjust_df(df)
    df.to_csv('data/{}.csv'.format(name_investment))
    print("save finishd {}".format(name_investment))
    return browser



target_url = "https://www.rakuten-sec.co.jp/ITS/V_ACT_Login.html"
target_list = ["ひふみプラス",
               "AI(人工知能)活用型世界株ファンド(ディープAI)",
               "eMAXIS Slim 先進国株式インデックス",
               "グローバルAIファンド",
               "グローバルAIファンド(為替ヘッジあり)",
               "楽天日本株4.3倍ブル",
               "ベトナム成長株インカムファンド"]

#ログイン画面
browser = webdriver.Chrome()
browser.get(target_url)
id = browser.find_element_by_id("form-login-id")
id.send_keys(ID)
password = browser.find_element_by_id("form-login-pass")
password.send_keys(PASSWORD)
time.sleep(1)
login_button = browser.find_element_by_id("login-btn")
login_button.click()
browser.find_element_by_xpath('//*[@id="gmenu_invest"]/a').click()
time.sleep(1)

#確認画面
try:
    browser.find_element_by_class_name('modal__button--normal').click()
except:
    pass
time.sleep(1)
browser.find_element_by_xpath('//*[@id="nav-sub-menu"]/ul/li[3]/a').click()




In [4]:
#スーパーサーチの画面
from selenium.webdriver.common.keys import Keys

target_dic = {}
num_target_page=50


for t in range(num_target_page):
    #テーブルの状態を取得
    tbl_element = browser.find_element_by_css_selector('table.tbl-data-02.tbl-stripe-01')
    trs = tbl_element.find_elements_by_tag_name("tr")


    for i in range(3,len(trs)):
        print(i)
        tds = trs[i].find_elements(By.TAG_NAME, "th")
        for j in tds:

            print(j.text)
            name_investment = j.text
            print(j.find_element_by_css_selector("a").get_attribute("href"))
            
            j.find_element_by_css_selector("a").click()

            #各投資信託過去ページ
            try:
                browser.find_elements_by_class_name("segment")[3].find_elements_by_class_name("link-01")[0].find_element_by_css_selector("a").click()
                #ブラウザ切り替える
                window = browser.window_handles[1]
                browser.switch_to.window(window)
                browser = get_data_investment(browser, name_investment=name_investment)
                window = browser.window_handles[0]
                browser.close()
                browser.switch_to.window(window)
            
            except:
                print("skipped {}".format(name_investment))
                
           
       
            #ブラウザを戻す
            browser.back()
            time.sleep(1)
            
            #テーブルの状態を再度取得
            tbl_element = browser.find_element_by_css_selector('table.tbl-data-02.tbl-stripe-01')
            trs = tbl_element.find_elements_by_tag_name("tr")

    browser.find_element_by_xpath('//*[@id="search-right-column-inner"]/ul[1]/li[3]/a').click()


3
楽天・全米株式インデックス・ファンド
https://member.rakuten-sec.co.jp/app/info_fu_invest_detail.do;BV_SessionID=A15EF6AE432E0AA1A462C1D7CA74A4DC.441732a5?eventType=init&SS_URL=detail/index.html&ID=JP90C000FHD2&r401k=true
No more pages
save finishd 楽天・全米株式インデックス・ファンド
4
ひふみプラス
https://member.rakuten-sec.co.jp/app/info_fu_invest_detail.do;BV_SessionID=A15EF6AE432E0AA1A462C1D7CA74A4DC.441732a5?eventType=init&SS_URL=detail/index.html&ID=JP90C0008CH5
No more pages
skipped ひふみプラス


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"table.tbl-data-02.tbl-stripe-01"}
  (Session info: chrome=69.0.3497.100)
  (Driver info: chromedriver=2.41.578706 (5f725d1b4f0a4acbf5259df887244095596231db),platform=Mac OS X 10.13.3 x86_64)


In [ ]:
browser.find_elements_by_class_name("segment")[3].find_elements_by_class_name("link-01")[0].find_element_by_css_selector("a").click()

In [ ]:
trs[3].find_elements(By.TAG_NAME, "td")

In [ ]:
import pickle
with open('target_dic.pickle', 'wb') as f:
    pickle.dump(target_dic, f)
    
with open('target_dic.pickle', mode='rb') as fi:
    data = pickle.load(fi)

In [ ]:
for name,url in data.items():
    print(url)
browser = webdriver.Chrome()
browser.get(url)

In [ ]:


def login(browser,target_url):
    browser = webdriver.Chrome()
    browser.get(target_url)
    id = browser.find_element_by_id("form-login-id")
    id.send_keys(ID)
    password = browser.find_element_by_id("form-login-pass")
    password.send_keys(PASSWORD)
    time.sleep(1)
    login_button = browser.find_element_by_id("login-btn")
    login_button.click()
    return browser

target_url = "https://www.rakuten-sec.co.jp/ITS/V_ACT_Login.html"
browser = login(browser,target_url)


for name,url in data.items():
    browser.get(url)  
    browser.find_element_by_xpath('//*[@id="infoPage"]/div[6]/div[2]/p[1]/a').click()
    
    window = browser.window_handles[1]
    browser.switch_to.window(window)
    
    tbl_element = browser.find_element_by_class_name('tbl-data-02')
    trs = tbl_element.find_elements_by_tag_name("tr")
    data_dic = {"data":{}}
    for i in range(1,len(trs)):
        tds = trs[i].find_elements(By.TAG_NAME, "td")
        try:
            data_dic["data"]={}
            data_dic["data"]["基準価額"]=tds[0].text
            data_dic["data"]["前日比"]=tds[1].text
            data_dic["data"]["分配金"]=tds[2].text
            data_dic["data"]["純資産総額"]=tds[3].text
            print(data_dic)
        except:
            pass


In [ ]:



class SearchMercariData(object):
    def __init__(self,input_data):
        self.input_data = input_data
        self.num_page = 1

    def search_from_sold_list(self):
        df_url = self.get_url(self.num_page)
        df = self.get_details(df_url)
        return df

    def get_url(self, num_page):
        browser = webdriver.Chrome()
        browser.get("https://www.mercari.com/jp/search/?keyword=")
        browser.find_element_by_xpath("/html/body/div/main/div[2]/form/div[2]/div[8]/div/div[2]/label").click()
        browser.find_element_by_xpath("/html/body/div/main/div[2]/form/div[2]/div[10]/button").click()
        df = pd.DataFrame()
        page=0
        for i in range(num_page):
            if len(browser.find_elements_by_css_selector("li.pager-next .pager-cell:nth-child(1) a")) > 0:
                print("######################page: {} ########################".format(page))
                print("Starting to get posts...")
                posts = browser.find_elements_by_css_selector(".items-box")
                for post in posts:
                    url = post.find_element_by_css_selector("a").get_attribute("href")
                    box_photo = post.find_element_by_css_selector(".items-box-photo")
                    pic = box_photo.find_element_by_css_selector("img").get_attribute("data-src")
                    se = pd.Series([url,pic],index=['url','pic'])
                    df = df.append(se, ignore_index=True)
                page+=1

                btn = browser.find_element_by_css_selector("li.pager-next .pager-cell:nth-child(1) a").get_attribute("href")
                print("next url:{}".format(btn))

                df = df.drop_duplicates()
                browser.get(btn)
                print("Moving to next page......")
            else:
                print("no pager exist anymore")
        return df


    def get_details(self,df_sold_out):
        df_details_list = pd.DataFrame()
        browser = webdriver.Chrome()
        for i in range(df_sold_out.shape[0]):
            url  = df_sold_out["url"].values[i]
            pic  = df_sold_out["pic"].values[i]
            browser.get(url)
            try:
                item_name =browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/h2").text
                try:
                    sold_badge =browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[1]/div/div[2]/div").text
                except:
                    sold_badge = "NOT_SOLD"
                main_category =browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[1]/table/tbody/tr[2]/td/a[1]/div").text
                sub1_category =browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[1]/table/tbody/tr[2]/td/a[2]/div").text
                sub2_category = browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[1]/table/tbody/tr[2]/td/a[3]/div").text
                brand_name = browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[1]/table/tbody/tr[3]/td").text

                box_details = browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[1]/table").text
                list_item_condition = ["新品、未使用","未使用に近い","目立った傷や汚れなし","やや傷や汚れあり","傷や汚れあり"]
                box_details = box_details.replace("\n"," ").split(" ")

                for word in list_item_condition:
                    try:
                        item_condition = box_details[box_details.index(word)]
                        break
                    except:
                        next
                shipping_fee =  browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[2]/span[3]").text
                item_description= browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[3]").text
                price = browser.find_element_by_xpath("/html/body/div[1]/main/div[1]/section[1]/div[2]/span[1]").text

                item_description = item_description.replace("\n"," ")
                price = price.replace("¥ ","")
                se = pd.Series([item_name, sold_badge, main_category,sub1_category,sub2_category,brand_name,item_condition,shipping_fee,item_description,price,url,pic],
                                   ["item_name", "sold_badge","main_category","sub1_category","sub2_category","brand_name","item_condition","shipping_fee","item_description","price","url","pic"])
                df_details_list = df_details_list.append(se, ignore_index=True)
                print("{} details of {} was added".format(i,url) )
            except:
                print("{} are not found".format(url))
        return df_details_list